Imports

In [ ]:
import numpy as np
import cvxpy as cp

## Exercise 1 - Investment Returns

Two investments are made, with random returns $R_1$ and $R_2$. The total return for the two investments is $R_1 + R_2$, and the probability of a loss (including breaking even, i.e., $R_1+R_2=0$) is $p^{loss} = \textbf{prob}(R_1 + R_2 \le 0)$. The goal is to find the worst-case (i.e., maximum possible) value of $p^{loss}$ consistent with the following information.

We discretize the values that $R_1$ and $R_2$ can take to $n=100$ possible values $r_1$, ..., $r_n$, uniformly spaced from $r_1 = -35$ to $r_n = +75$. $R_1$ and $R_2$ are assumed to have Gaussian [marginal distributions](https://en.wikipedia.org/wiki/Marginal_distribution) with known means $\mu_1$ and $\mu_2$ and known standard deviations $\sigma_1$ and $\sigma_2$. Hence, the discretized marginals $p^{(1)}$ and $p^{(2)}$ for $R_1$ and $R_2$ are given by

$$
\displaystyle p_i^{(k)} = \textbf{prob}(R_k = r_i) = \frac{\displaystyle e^{ \frac{-(r_i - \mu_k)^2}{2\sigma_k^2}}}{\displaystyle \sum_{j=1}^n e^{ \frac{-(r_j - \mu_k)^2}{2\sigma_k^2}}}
$$

for $k = 1,2$ and $i = 1, ..., n$. In addition it is known that $R_1$ and $R_2$ are correlated with [correlation coefficient](https://en.wikipedia.org/wiki/Correlation#Pearson's_product-moment_coefficient) $\rho$, which means that

$$
\sum_{i,j} (r_i - \mu_1) \cdot \textbf{prob}(R_1 = r_i , R_2 = r_j) \cdot (r_j - \mu_2) = \rho \sigma_1 \sigma_2
$$


Consider the data $\mu_1 =9$, $\mu_2 = 22$, $\sigma_1 = 7$, $\sigma_2 = 19$ and $\rho = -0.35$. 



In [ ]:
import cvxpy as cp
import numpy as np

# Data
n = 100
rmin = -35 
rmax = 75

mu1 = 9 
mu2 = 22
sigma1 = 7 
sigma2 = 19
rho = -0.35

Formulate the problem as a convex optimisation problem and solve it.

## Solution 1


In [ ]:
# Maximum ploss corresponds to minimum pwin(R1 + R2 > 0)
#Generate likelihood matrices for R1 and R2

x = np.linspace(rmin,rmax,n)
p1 = np.exp(-((x-mu1)**2)/(2*sigma1**2))
p2 = np.exp(-((x-mu2)**2)/(2*sigma2**2))

p1 = p1/p1.sum()
p2 = p2/p2.sum()

# This is our R1 + R2 matrix 
X = np.zeros((n,n))
for i in range(n):
  for k in range(n):
    X[i][k] = x[i] +x[k]



In [ ]:
# constraint is  = 
b = np.nonzero(X<=0) # index of nonzero elements 

P = cp.Variable((n,n))

objective = cp.Maximize(cp.sum(P[b]))

constraints = [(x-mu1) @ P @(x-mu2) == sigma1*sigma2*rho
               ,cp.sum(P) == 1
               ,P >= 0
              ,cp.sum(P,1) == p1
               ,cp.sum(P,0) == p2

               ]

# every p is bigger than zero

prob = cp.Problem(objective,constraints)



In [ ]:
prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.2.3                                    
(CVXPY) Jan 27 12:08:45 PM: Your problem has 10000 variables, 5 constraints, and 0 parameters.
(CVXPY) Jan 27 12:08:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 27 12:08:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 27 12:08:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 27 12:08:45 PM: Compiling problem (target solver=ECOS).
(CVXPY) Jan 27 12:08:45 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> C

0.18928791828101693

In [ ]:
(x-mu1).shape

(100,)